In [26]:
import polars as pl

schema_overrides = {
    "unique_key": pl.Int64,
    "created_date": pl.Datetime,
    "closed_date": pl.Datetime,
    "agency": pl.Utf8,
    "complaint_type": pl.Utf8,
    "descriptor": pl.Utf8,
    "location_type": pl.Utf8,
    "incident_zip": pl.Utf8,
    "borough": pl.Utf8,
    "open_data_channel_type": pl.Utf8,
    "latitude": pl.Float64,
    "longitude": pl.Float64,
    "community_board": pl.Utf8,
    "response_time_hours": pl.Float64,
    "response_time_minutes": pl.Float64,
    "response_time_seconds": pl.Float64,

}

df = pl.read_csv(
    "../NYC_311_Data/outputs/nyc_311_cleaned.csv",
    schema_overrides=schema_overrides,
    null_values=["NA", "N/A", "null", ""],  # Treat these as missing
    infer_schema_length=1000000
)


## Temporal Features
This section extracts temporal features from created_date to capture time-based patterns:

1. day_of_week: Day of the week (1 = Monday, 7 = Sunday).
2. hour: Hour of the day (0-23).
3. month: Month of the year (1-12).
4. week_number: ISO week number of the year
4. is_weekend: Binary feature (1 if Saturday or Sunday, 0 otherwise).
5. is_rush_hour: Binary feature (1 if time is 7-9 AM or 4-6 PM on weekdays, 0 otherwise).
6. is_holiday: Check if a complaint date is a US holiday
7. season: Winter, Spring, Summer, Fall
8. time_of_day: Night, Morning, Afternoon, Evening 

#### Day of week, Hour and month

In [27]:
# Extract day of week, hour, and month
df = df.with_columns([
    pl.col('created_date').dt.weekday().alias('day_of_week'),
    pl.col('created_date').dt.hour().alias('hour'),
    pl.col('created_date').dt.month().alias('month'),
    pl.col('created_date').dt.week().alias('week_number')
])

#### is weekend or is weekday

In [28]:
# Create binary feature for weekends (Saturday and Sunday)
df = df.with_columns((pl.col('day_of_week') >= 6).alias('is_weekend'))

#### rush hour

In [29]:
# Create binary feature for rush hours (7-9 AM and 4-6 PM on weekdays)
df = df.with_columns(
    ((pl.col('hour').is_in([7, 8, 9, 16, 17, 18])) & (pl.col('day_of_week') <= 5)).alias('is_rush_hour')
)

#### is holiday

In [30]:
import holidays
us_holidays = holidays.US()
min_date = df["created_date"].min().date()
max_date = df["created_date"].max().date()

# Determine the range of years from your dataset
years = list(range(min_date.year, max_date.year + 1))
us_holidays = holidays.US(years=years)
holiday_keys = list(us_holidays.keys())

holiday_dates = [date for date in holiday_keys if min_date <= date <= max_date]

# Add an 'is_holiday' column to the DataFrame
df = df.with_columns(
    pl.col("created_date").dt.date().is_in(holiday_dates).alias("is_holiday")
)

#### Season

In [31]:
df = df.with_columns(
    pl.when(pl.col("month").is_in([12, 1, 2])).then(pl.lit("Winter"))
    .when(pl.col("month").is_in([3, 4, 5])).then(pl.lit("Spring"))
    .when(pl.col("month").is_in([6, 7, 8])).then(pl.lit("Summer"))
    .when(pl.col("month").is_in([9, 10, 11])).then(pl.lit("Fall"))
    .otherwise(pl.lit("Unknown")).alias("season")
)

#### Time of day

In [32]:
df = df.with_columns(
    pl.when((pl.col("hour") >= 0) & (pl.col("hour") < 6)).then(pl.lit("Night"))
    .when((pl.col("hour") >= 6) & (pl.col("hour") < 12)).then(pl.lit("Morning"))
    .when((pl.col("hour") >= 12) & (pl.col("hour") < 18)).then(pl.lit("Afternoon"))
    .when((pl.col("hour") >= 18) & (pl.col("hour") < 24)).then(pl.lit("Evening"))
    .otherwise(pl.lit("Unknown")).alias("time_of_day")
)

## Spatial Features
This section creates spatial features to capture geographic patterns:

1. Borough Dummies: One-hot encoded columns for borough (e.g., 'borough_Manhattan', 'borough_Brooklyn').
2. zip_complaint_freq: Frequency of complaints per incident_zip.

In [33]:
# One-hot encode 'borough'
borough_dummies = df['borough'].to_dummies()
df = df.hstack(borough_dummies)
del borough_dummies

In [34]:
# Calculate complaint frequency per zip code
zip_freq = df.group_by('incident_zip').agg(pl.len().alias('zip_complaint_freq'))
df = df.join(zip_freq, on='incident_zip', how='left')

## Complaint Characteristics Features
This section encodes complaint-related categorical columns using frequency encoding to handle high cardinality:

1. agency_freq: Frequency of each agency.
2. complaint_type_freq: Frequency of each complaint_type.
3. descriptor_freq: Frequency of each descriptor.

In [35]:
# Frequency encoding for 'agency'
agency_freq = df.group_by('agency').agg(pl.len().alias('agency_freq'))
df = df.join(agency_freq, on='agency', how='left')

# Frequency encoding for 'complaint_type'
complaint_type_freq = df.group_by('complaint_type').agg(pl.len().alias('complaint_type_freq'))
df = df.join(complaint_type_freq, on='complaint_type', how='left')


# Frequency encoding for 'descriptor'
descriptor_freq = df.group_by('descriptor').agg(pl.len().alias('descriptor_freq'))
df = df.join(descriptor_freq, on='descriptor', how='left')

## Rolling Averages

1. Daily Complaint Counts (7-day and 30-day): This shows recent trends in complaint volume, useful for workload analysis.
2. Response Times (7-day and 30-day): This tracks efficiency trends, helping identify periods of slower or faster resolutions.
3. Complaint Counts by Type (7-day and 30-day): This captures trends for specific complaint types, like noise complaints increasing over time.
4. Complaint Counts by Borough (7-day and 30-day): This shows borough-specific trends, like more complaints in Manhattan during certain periods.

In [36]:
df = df.with_columns(pl.col("created_date").dt.date().alias("date"))

#### Daily complaint counts

In [37]:
daily_counts = df.group_by("date").agg(pl.len().alias("daily_complaints"))
daily_counts = daily_counts.sort("date").with_columns(
    pl.col("daily_complaints").rolling_mean(window_size=7).alias("7_day_avg_complaints"),
    pl.col("daily_complaints").rolling_mean(window_size=30).alias("30_day_avg_complaints")
)
df = df.join(daily_counts, on="date", how="left")

#### Response times

In [38]:
daily_response = df.group_by("date").agg(pl.col("response_time_hours").mean().alias("daily_avg_response"))
daily_response = daily_response.sort("date").with_columns(
    pl.col("daily_avg_response").rolling_mean(window_size=7).alias("7_day_avg_response"),
    pl.col("daily_avg_response").rolling_mean(window_size=30).alias("30_day_avg_response")
)
df = df.join(daily_response, on="date", how="left")

#### Complaint Counts by Type

In [ ]:
daily_type_counts = df.group_by(["date", "complaint_type"]).agg(pl.len().alias("daily_type_complaints"))
rolling_type_counts = daily_type_counts.sort(["complaint_type", "date"]).group_by("complaint_type").map_groups(
    lambda group: group.with_columns(
        pl.col("daily_type_complaints").rolling_mean(window_size=7).alias("7_day_avg_type_complaints"),
        pl.col("daily_type_complaints").rolling_mean(window_size=30).alias("30_day_avg_type_complaints")
    )
)
df = df.join(rolling_type_counts, on=["date", "complaint_type"], how="left")

#### Complaint Counts by Borough

In [40]:
daily_borough_counts = df.group_by(["date", "borough"]).agg(pl.len().alias("daily_borough_complaints"))
rolling_borough_counts = daily_borough_counts.sort(["borough", "date"]).group_by("borough").map_groups(
    lambda group: group.with_columns(
        pl.col("daily_borough_complaints").rolling_mean(window_size=7).alias("7_day_avg_borough_complaints"),
        pl.col("daily_borough_complaints").rolling_mean(window_size=30).alias("30_day_avg_borough_complaints")
    )
)
df = df.join(rolling_borough_counts, on=["date", "borough"], how="left")

## Add Weather data

In [41]:
# Step 1: Load the weather data
weather_df = pl.read_csv("../Weather_data/nyc_weather.csv", schema_overrides={
    'DATE': pl.Date, 
    'PRCP': pl.Utf8, 
    'SNOW': pl.Utf8, 
    'SNWD': pl.Utf8, 
    'TMAX': pl.Utf8, 
    'TMIN': pl.Utf8
})

In [42]:
# Step 2: Define wt_mapping dictionary (based on NOAA GHCN daily documentation)
wt_mapping = {
    'WT01': 'fog',
    'WT02': 'heavy_fog',
    'WT03': 'thunder',
    'WT04': 'ice_pellets',
    'WT05': 'hail',
    'WT06': 'glaze',
    'WT07': 'dust',
    'WT08': 'smoke_haze',
    'WT09': 'blowing_snow',
    'WT10': 'tornado',
    'WT11': 'high_winds',
    'WT12': 'blowing_spray',
    'WT13': 'mist',
    'WT14': 'drizzle',
    'WT15': 'freezing_drizzle',
    'WT16': 'rain',
    'WT17': 'freezing_rain',
    'WT18': 'snow',
    'WT19': 'unknown_precip',
    'WT21': 'ground_fog',
    'WT22': 'ice_fog'
}

# Step 3: Identify WT columns in the dataset
wt_cols = [col for col in weather_df.columns if col.startswith('WT') and not col.endswith('_ATTRIBUTES')]

# Step 4: Process the weather data
weather_df = weather_df.with_columns([
    # Handle trace values (replace "T" with 0.01) and cast to numeric
    pl.col('PRCP').str.replace("T", "0.01").cast(pl.Float64).alias('precipitation'),
    pl.col('SNOW').str.replace("T", "0.01").cast(pl.Float64).alias('snowfall'),
    pl.col('SNWD').str.replace("T", "0.01").cast(pl.Float64).alias('snow_depth'),
    pl.col('TMAX').cast(pl.Float64).alias('max_temperature'),
    pl.col('TMIN').cast(pl.Float64).alias('min_temperature'),
    # Process WT columns: convert to 0/1 indicators after removing whitespace
    *[pl.col(col).str.strip_chars().eq("1").cast(pl.Int8).fill_null(0).alias(wt_mapping.get(col, col.lower())) 
      for col in wt_cols]
])

In [43]:

# Step 5: Drop original WT columns and unnecessary columns
columns_to_drop = [col for col in weather_df.columns if col.startswith('WT')]
weather_df = weather_df.drop(columns_to_drop)


In [44]:

# Step 6: Select only the relevant columns
weather_df = weather_df.select([
    'DATE', 'precipitation', 'snowfall', 'snow_depth', 'max_temperature', 'min_temperature'] + 
    [wt_mapping[col] for col in wt_cols if col in wt_mapping]
)

In [45]:
# Step 7: Merge with your main DataFrame
# Assuming your main DataFrame is called 'df' and has a 'created_date' column
df = df.with_columns(pl.col('created_date').dt.date().alias('date'))  # Extract date from created_date
df = df.join(weather_df, left_on='date', right_on='DATE', how='left')   # Left join to keep all rows in df

## Operational Workload feature

#### 1. Agency backlog: Number of open requests per agency at time of filing 
#### Create daily counts of opened and closed requests per agency

In [46]:
daily_agency_tracker = df.select([
    pl.col("agency"),
    pl.col("created_date").dt.date().alias("date"),
    pl.lit(1).alias("opened"),
    pl.lit(0).alias("closed")
]).vstack(
    df.select([
        pl.col("agency"),
        pl.col("closed_date").dt.date().alias("date"),
        pl.lit(0).alias("opened"),
        pl.lit(1).alias("closed")
    ])
)

In [47]:
# Group by agency and date, sum the opened and closed counts
daily_agency_tracker = daily_agency_tracker.group_by(["agency", "date"]).agg([
    pl.sum("opened").alias("opened"),
    pl.sum("closed").alias("closed")
])

# Sort by agency and date
daily_agency_tracker = daily_agency_tracker.sort(["agency", "date"])

# Calculate running totals (backlog) for each agency
daily_agency_tracker = daily_agency_tracker.group_by("agency").map_groups(
    lambda group: group.with_columns([
        (pl.col("opened").cum_sum() - pl.col("closed").cum_sum()).alias("agency_backlog")
    ])
)


In [48]:

# Join backlog to main dataframe
df = df.with_columns(pl.col("created_date").dt.date().alias("request_date"))
df = df.join(
    daily_agency_tracker.select(["agency", "date", "agency_backlog"]),
    left_on=["agency", "request_date"],
    right_on=["agency", "date"],
    how="left"
)

####  2. Concurrent requests: Number of similar requests filed in the same timeframe 
#### We'll create both daily and hourly counts

In [49]:

df = df.with_columns([
    pl.col("created_date").dt.date().alias("created_date_only"),
    pl.col("created_date").dt.hour().alias("created_hour")
])

# Daily concurrent complaints of the same type
daily_concurrent = df.group_by(["complaint_type", "created_date_only"]).agg([
    pl.len().alias("daily_concurrent_count")
])

# Hourly concurrent complaints of the same type
hourly_concurrent = df.group_by(["complaint_type", "created_date_only", "created_hour"]).agg([
    pl.len().alias("hourly_concurrent_count")
])

# Join both back to the main dataframe
df = df.join(daily_concurrent, on=["complaint_type", "created_date_only"], how="left")
df = df.join(hourly_concurrent, on=["complaint_type", "created_date_only", "created_hour"], how="left")


C:\Users\ronyr\AppData\Local\Temp\ipykernel_27240\1099281293.py:8: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("daily_concurrent_count")
C:\Users\ronyr\AppData\Local\Temp\ipykernel_27240\1099281293.py:13: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("hourly_concurrent_count")


####  3. Previous day completion rate

In [50]:
# Calculate daily completion rate
daily_completion = df.filter(pl.col("closed_date").is_not_null()).group_by(
    pl.col("closed_date").dt.date().alias("date")
).agg([
    pl.len().alias("closed_count")
])

# Get daily opening count
daily_opening = df.group_by(
    pl.col("created_date").dt.date().alias("date")
).agg([
    pl.len().alias("opened_count")
])

# Join and calculate rate
daily_stats = daily_opening.join(daily_completion, on="date", how="full").fill_null(0)
daily_stats = daily_stats.sort("date").with_columns([
    (pl.col("closed_count") / pl.col("opened_count").shift(1)).fill_null(0).alias("prev_day_completion_rate")
])

# Join back to main dataframe
df = df.join(
    daily_stats.select(["date", "prev_day_completion_rate"]), 
    left_on="created_date_only", 
    right_on="date", 
    how="left"
)

C:\Users\ronyr\AppData\Local\Temp\ipykernel_27240\1547394833.py:16: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  daily_stats = daily_opening.join(daily_completion, on="date", how="outer").fill_null(0)


#### 4. Spatial clustering

In [53]:
if all(col in df.columns for col in ["latitude", "longitude"]):
    # Create grid cells for spatial binning (approximately 500m grid at NYC latitude)
    grid_size = 0.005  # ~500m at NYC latitude

    df = df.with_columns([
        pl.when(pl.col("latitude").is_not_null() & pl.col("longitude").is_not_null())
        .then((pl.col("latitude") / grid_size).floor() * grid_size)  # Use .floor() here
        .otherwise(None)
        .alias("lat_grid"),

        pl.when(pl.col("latitude").is_not_null() & pl.col("longitude").is_not_null())
        .then((pl.col("longitude") / grid_size).floor() * grid_size)  # Use .floor() here
        .otherwise(None)
        .alias("lon_grid")
    ])

    # Count complaints in each grid cell
    grid_density = df.filter(
        pl.col("lat_grid").is_not_null() & pl.col("lon_grid").is_not_null()
    ).group_by(["lat_grid", "lon_grid"]).agg([
        pl.len().alias("grid_complaint_density")
    ])

    
    # Join back to main dataframe
    df = df.join(grid_density, on=["lat_grid", "lon_grid"], how="left")

#### 5. Public safety indicator

In [54]:
# Safety-related complaint types (based on the list analysis)
safety_related_types = [
    "Safety", "Illegal Fireworks", "Unstable Building", "Facade Insp Safety Pgm", 
    "Facades", "Structural", "Forensic Engineering", "Asbestos", "Hazardous Materials", 
    "Lead", "Oil Or Gas Spill", "Radioactive Material", "Emergency Response Team (Ert)", 
    "Scaffold Safety", "Dep Highway Condition", "Air Quality", "Indoor Air Quality", 
    "X-Ray Machine/Equipment", "Cooling Tower", "Drinking Water", "Water Quality", 
    "Building Drinking Water Tank", "Indoor Sewage", "Food Poisoning", "Highway Sign - Dangling", 
    "Street Sign - Dangling", "Building Condition", "Face Covering Violation", 
    "Vaccine Mandate Non-Compliance", "Private School Vaccine Mandate Non-Compliance",
    "Traffic Signal Condition", "Elevator", "Plumbing", "Sewer", "Water Leak", 
    "Water System", "Sewer Maintenance", "Electric", "Electrical", "Flooring/Stairs", 
    "Door/Window", "Construction Safety Enforcement"
]

# Convert to lowercase for case-insensitive matching
safety_related_types_lower = [complaint_type.lower() for complaint_type in safety_related_types]

# Create safety indicator
df = df.with_columns([
    pl.col("complaint_type").str.to_lowercase().is_in(safety_related_types_lower).alias("is_safety_related")
])

# Create a weighted safety score (1-5) for different categories of safety concerns
high_priority_safety = ["Unstable Building", "Emergency Response Team (Ert)", 
                       "Hazardous Materials", "Oil Or Gas Spill", "Radioactive Material"]
high_priority_lower = [item.lower() for item in high_priority_safety]

df = df.with_columns([
    pl.when(pl.col("complaint_type").str.to_lowercase().is_in(high_priority_lower))
    .then(5)  # Highest priority
    .when(pl.col("is_safety_related"))
    .then(3)  # Medium priority
    .otherwise(1)  # Standard priority
    .alias("safety_priority_score")
])

####  6. Complaint priority score based on historical agency response patterns

In [55]:

# Calculate average response time per complaint type
complaint_priority = df.group_by("complaint_type").agg([
    pl.mean("response_time_hours").alias("avg_response_time"),
    pl.std("response_time_hours").alias("std_response_time"),
    pl.len().alias("complaint_count")
])

# Calculate priority score (inverse of response time - faster response = higher priority)
complaint_priority = complaint_priority.with_columns([
    (1 / (pl.col("avg_response_time") + 0.001)).alias("raw_priority_score")
])

# Normalize to 0-1 scale
complaint_priority = complaint_priority.with_columns([
    (pl.col("raw_priority_score") / pl.col("raw_priority_score").max()).alias("priority_score")
])

# Join back to main dataframe
df = df.join(
    complaint_priority.select(["complaint_type", "priority_score"]),
    on="complaint_type",
    how="left"
)

#### 7. Request escalation indicator 

In [56]:

# Identify complaints that typically take much longer than average to resolve
escalation_threshold = df.group_by("complaint_type").agg([
    pl.mean("response_time_hours").alias("mean_response"),
    pl.std("response_time_hours").alias("std_response")
]).with_columns([
    (pl.col("mean_response") + 2 * pl.col("std_response")).alias("escalation_threshold")
])

# Join back to main dataframe
df = df.join(
    escalation_threshold.select(["complaint_type", "escalation_threshold"]),
    on="complaint_type",
    how="left"
)

# Mark complaints that exceed threshold
df = df.with_columns([
    (pl.col("response_time_hours") > pl.col("escalation_threshold")).alias("is_likely_escalated")
])

# Calculate historical escalation rate by complaint type
escalation_rate = df.group_by("complaint_type").agg([
    pl.mean("is_likely_escalated").cast(pl.Float32).alias("historical_escalation_rate")
])

# Join back to main dataframe
df = df.join(
    escalation_rate,
    on="complaint_type",
    how="left"
)

## Advanced Temporal features

In [58]:
# Time to end of business day (hours remaining until 5pm)
df = df.with_columns([
    pl.when(
        (pl.col("hour") < 17) & (pl.col("day_of_week") < 5)  # Before 5pm on weekdays
    ).then(
        17 - pl.col("hour")
    ).otherwise(
        # If after 5pm or weekend, set to 0
        pl.lit(0)
    ).alias("hours_to_end_of_business")
])

# Days until weekend
df = df.with_columns([
    pl.when(pl.col("day_of_week") < 5)  # Weekday (0-4 = Mon-Fri)
    .then(5 - pl.col("day_of_week"))  # Days until Saturday
    .otherwise(pl.lit(0))  # Already weekend
    .alias("days_to_weekend")
])

# Days until next holiday
# This is more complex - need to calculate for each date
from datetime import timedelta

# Get all dates in dataframe
unique_dates = df.select(pl.col("created_date").dt.date()).unique().to_series()
unique_dates_list = unique_dates.to_list()

In [59]:


# Calculate days to next holiday for each date
days_to_holiday_map = {}
for date in unique_dates_list:
    # Find next holiday
    next_holiday = None
    days_diff = float('inf')
    
    for holiday in holiday_dates:
        if holiday > date:
            curr_days = (holiday - date).days
            if curr_days < days_diff:
                days_diff = curr_days
                next_holiday = holiday
    
    days_to_holiday_map[date] = days_diff if days_diff != float('inf') else 0


In [60]:

# Create mapping dataframe
holiday_mapping = pl.DataFrame({
    "date": list(days_to_holiday_map.keys()),
    "days_to_next_holiday": list(days_to_holiday_map.values())
})


In [61]:

# Join to main dataframe
df = df.join(
    holiday_mapping,
    left_on=pl.col("created_date").dt.date(),
    right_on="date",
    how="left"
)

##  Interaction Features

#### Weather severity x complaint type interactions

In [62]:

# Create a weather severity index
df = df.with_columns([
    (pl.col("precipitation") * 5 + 
     pl.col("snowfall") * 3 + 
     pl.col("high_winds").cast(pl.Float64) * 4 +
     pl.col("thunder").cast(pl.Float64) * 3 +
     ((pl.col("max_temperature") > 90).cast(pl.Float64) * 3) +
     ((pl.col("min_temperature") < 20).cast(pl.Float64) * 3)
    ).alias("weather_severity_index")
])


#### Create interaction between weather severity and complaint types

In [63]:

# First group complaint types into categories that might be weather-sensitive
weather_sensitive_complaints = [
    "Street Condition", "Flooding", "Water System", "Sewer", "Snow", 
    "Water Leak", "Street Light Condition", "Heat/Hot Water"
]
weather_sensitive_complaints_lower = [c.lower() for c in weather_sensitive_complaints]

df = df.with_columns([
    pl.col("complaint_type").str.to_lowercase().is_in(weather_sensitive_complaints_lower).alias("is_weather_sensitive")
])

# Create interaction feature
df = df.with_columns([
    (pl.col("weather_severity_index") * pl.col("is_weather_sensitive").cast(pl.Float64)).alias("weather_complaint_interaction")
])

#### Weekend x agency interaction

In [64]:
# Group agencies by those that likely operate differently on weekends
emergency_agencies = ["FDNY", "NYPD", "DOHMH", "DEP"]
df = df.with_columns([
    (pl.col("is_weekend") & pl.col("agency").is_in(emergency_agencies)).alias("weekend_emergency_agency"),
    (pl.col("is_weekend") & ~pl.col("agency").is_in(emergency_agencies)).alias("weekend_non_emergency_agency")
])

#### Seasonal agency workload

In [65]:
seasons = ["Winter", "Spring", "Summer", "Fall"]
for season in seasons:
    season_lower = season.lower()
    # Create indicator for each season-agency combination
    df = df.with_columns([
        ((pl.col("season") == season) & (pl.col("agency") == ag)).alias(f"{season_lower}_{ag}")
        for ag in df["agency"].unique().to_list()[:10]  # Limit to top 10 agencies to avoid explosion of features
    ])

### Covid indicator

In [66]:
# COVID-19 period indicators
df = df.with_columns([
    # Pre-COVID (before March 1, 2020)
    (pl.col("created_date").dt.date() < pl.date(2020, 3, 1)).alias("pre_covid"),
    
    # Initial lockdown phase (March-June 2020)
    ((pl.col("created_date").dt.date() >= pl.date(2020, 3, 1)) & 
     (pl.col("created_date").dt.date() < pl.date(2020, 7, 1))).alias("covid_lockdown"),
    
    # Phased reopening (July-Nov 2020)
    ((pl.col("created_date").dt.date() >= pl.date(2020, 7, 1)) & 
     (pl.col("created_date").dt.date() < pl.date(2020, 12, 1))).alias("covid_reopening"),
    
    # Winter surge (Dec 2020-Feb 2021)
    ((pl.col("created_date").dt.date() >= pl.date(2020, 12, 1)) & 
     (pl.col("created_date").dt.date() < pl.date(2021, 3, 1))).alias("covid_winter_surge"),
    
    # Delta variant period (Jun-Nov 2021)
    ((pl.col("created_date").dt.date() >= pl.date(2021, 6, 1)) & 
     (pl.col("created_date").dt.date() < pl.date(2021, 12, 1))).alias("covid_delta"),
    
    # Omicron variant period (Dec 2021-Feb 2022)
    ((pl.col("created_date").dt.date() >= pl.date(2021, 12, 1)) & 
     (pl.col("created_date").dt.date() < pl.date(2022, 3, 1))).alias("covid_omicron"),
    
    # Post-major restrictions (after March 2022)
    (pl.col("created_date").dt.date() >= pl.date(2022, 3, 1)).alias("post_major_covid")
])

### NYC School Calendar

In [67]:
# Approximate NYC school calendar indicators
df = df.with_columns([
    # Summer break (roughly late June through early September)
    ((pl.col("month").is_in([7, 8])) | 
     ((pl.col("month") == 6) & (pl.col("created_date").dt.day() >= 25)) |
     ((pl.col("month") == 9) & (pl.col("created_date").dt.day() <= 5))).alias("school_summer_break"),
    
    # Winter break (roughly Dec 24-Jan 2)
    (((pl.col("month") == 12) & (pl.col("created_date").dt.day() >= 24)) | 
     ((pl.col("month") == 1) & (pl.col("created_date").dt.day() <= 2))).alias("school_winter_break"),
    
    # Spring break (approximately first/second week of April)
    ((pl.col("month") == 4) & (pl.col("created_date").dt.day().is_between(1, 10))).alias("school_spring_break"),
])

# Create a general "school_in_session" indicator
df = df.with_columns([
    (~(pl.col("school_summer_break") | pl.col("school_winter_break") | 
       pl.col("school_spring_break") | pl.col("is_holiday")) & 
     (pl.col("day_of_week") < 5)).alias("school_in_session")
])

### NYC Major Events

In [68]:
# Major NYC events indicators
df = df.with_columns([
    # NYC Marathon (first Sunday in November)
    ((pl.col("month") == 11) & 
     (pl.col("day_of_week") == 6) & 
     (pl.col("created_date").dt.day() <= 7)).alias("nyc_marathon_day"),
    
    # Thanksgiving Parade (Thanksgiving Day)
    ((pl.col("month") == 11) & 
     (pl.col("day_of_week") == 3) & 
     (pl.col("created_date").dt.day() >= 22) & 
     (pl.col("created_date").dt.day() <= 28)).alias("thanksgiving_parade"),
    
    # New Year's Eve/Day 
    (((pl.col("month") == 12) & (pl.col("created_date").dt.day() == 31)) | 
     ((pl.col("month") == 1) & (pl.col("created_date").dt.day() == 1))).alias("new_years"),
    
    # St. Patrick's Day Parade (March 17 or nearest weekend)
    ((pl.col("month") == 3) & 
     (pl.col("created_date").dt.day().is_between(15, 19))).alias("st_patricks_parade"),
    
    # Pride March (last Sunday in June)
    ((pl.col("month") == 6) & 
     (pl.col("day_of_week") == 6) & 
     (pl.col("created_date").dt.day() >= 24)).alias("pride_march"),
])

### Weather and Complaint type interactive features

In [69]:
# Create weather severity indicators
df = df.with_columns([
    # Heavy rain day (precipitation > 1 inch)
    (pl.col("precipitation") > 1.0).alias("heavy_rain_day"),
    
    # Snow day
    (pl.col("snowfall") > 0.5).alias("snow_day"),
    
    # Extreme heat (max temp > 90°F)
    (pl.col("max_temperature") > 90).alias("extreme_heat_day"),
    
    # Extreme cold (min temp < 20°F)
    (pl.col("min_temperature") < 20).alias("extreme_cold_day")
])

# Map complaint types to categories that would be affected by different weather
# Heat-related complaints
df = df.with_columns([
    pl.col("complaint_type").str.to_lowercase().is_in([
        "heat/hot water", "residential heat", "cooling tower"
    ]).alias("is_heat_related_complaint")
])

# Water/flood-related complaints
df = df.with_columns([
    pl.col("complaint_type").str.to_lowercase().is_in([
        "sewer", "water system", "water leak", "flooding", "standing water", "plumbing"
    ]).alias("is_water_related_complaint")
])

# Street condition complaints
df = df.with_columns([
    pl.col("complaint_type").str.to_lowercase().is_in([
        "street condition", "street light condition", "sidewalk condition", 
        "highway condition", "traffic signal condition"
    ]).alias("is_street_related_complaint")
])

# Create interaction features
df = df.with_columns([
    (pl.col("heavy_rain_day") & pl.col("is_water_related_complaint")).alias("rain_water_interaction"),
    (pl.col("snow_day") & pl.col("is_street_related_complaint")).alias("snow_street_interaction"),
    (pl.col("extreme_heat_day") & pl.col("is_heat_related_complaint")).alias("heat_complaint_interaction"),
    (pl.col("extreme_cold_day") & pl.col("is_heat_related_complaint")).alias("cold_heat_complaint_interaction")
])

### Create weekend/holiday and complaint type interactions

In [70]:
df = df.with_columns([
    (pl.col("is_weekend") & pl.col("complaint_type").str.to_lowercase().is_in(["noise - residential", "noise - street/sidewalk", "noise - commercial"])).alias("weekend_noise_complaint"),
    
    (pl.col("is_holiday") & pl.col("complaint_type").str.to_lowercase().is_in(["noise - residential", "noise - street/sidewalk", "illegal parking", "blocked driveway"])).alias("holiday_noise_parking_complaint")
])

## Save

In [72]:
# save the final dataframe
df.write_csv("../NYC_311_Data/outputs/nyc_311_features.csv")

# other SKIP


In [ ]:

# Create borough binary features (one-hot encoding)
boroughs = ['MANHATTAN', 'BROOKLYN', 'QUEENS', 'BRONX', 'STATEN ISLAND']
for borough in boroughs:
    df = df.with_columns([
        (pl.col('borough') == borough).cast(pl.Int8).alias(f'borough_{borough.lower().replace(" ", "_")}')
    ])

# Calculate distances from borough centers (approximation)
# These are approximate lat/long coordinates for borough centers
borough_coords = {
    'MANHATTAN': (40.7831, -73.9712),
    'BROOKLYN': (40.6782, -73.9442),
    'QUEENS': (40.7282, -73.7949),
    'BRONX': (40.8448, -73.8648),
    'STATEN ISLAND': (40.5795, -74.1502),
    'NYC_CENTER': (40.7128, -74.0060)  # Manhattan downtown
}

# Create distance features using Haversine formula
# First create helper function to calculate distances
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return c * r

# Add distance features for each borough center
for borough, (lat, lon) in borough_coords.items():
    name = borough.lower().replace(" ", "_")
    df = df.with_columns([
        haversine_distance(
            pl.col('latitude'), 
            pl.col('longitude'), 
            lat, 
            lon
        ).alias(f'distance_to_{name}')
    ])

# ----------------------
# Complaint Characteristics
# ----------------------

# Create frequency features by complaint type and zip code
complaint_freq = df.group_by(['complaint_type']).agg(
    pl.count().alias('complaint_type_frequency')
)

zip_freq = df.group_by(['incident_zip']).agg(
    pl.count().alias('zip_frequency')
)

agency_freq = df.group_by(['agency']).agg(
    pl.count().alias('agency_frequency')
)

# Join frequency features back to the main dataframe
df = df.join(complaint_freq, on='complaint_type', how='left')
df = df.join(zip_freq, on='incident_zip', how='left')
df = df.join(agency_freq, on='agency', how='left')

# Calculate average response time for each complaint type & agency
# This is a form of target encoding
avg_response_by_complaint = df.group_by(['complaint_type']).agg(
    pl.mean('response_time_hours').alias('avg_response_time_by_complaint')
)

avg_response_by_agency = df.group_by(['agency']).agg(
    pl.mean('response_time_hours').alias('avg_response_time_by_agency')
)

avg_response_by_agency_complaint = df.group_by(['agency', 'complaint_type']).agg(
    pl.mean('response_time_hours').alias('avg_response_time_by_agency_complaint')
)

# Join average response times back to the main dataframe
df = df.join(avg_response_by_complaint, on='complaint_type', how='left')
df = df.join(avg_response_by_agency, on='agency', how='left')
df = df.join(avg_response_by_agency_complaint, on=['agency', 'complaint_type'], how='left')

# ----------------------
# Categorical Encoding
# ----------------------

# Target encoding for high-cardinality categorical features
# (We already did a simple version with the average response times above)

# For location_type, descriptor, and open_data_channel_type
# Smoothed target encoding to prevent overfitting
def smoothed_target_encoding(df, col, target, min_samples=100):
    """Smoothed target encoding for a categorical feature"""
    global_mean = df.select(pl.mean(target)).collect()[0, 0]
    
    # Calculate the mean target for each category
    cat_stats = df.group_by(col).agg([
        pl.mean(target).alias(f'mean_{target}'),
        pl.count().alias('count')
    ])
    
    # Calculate the smoothed mean
    cat_stats = cat_stats.with_columns([
        ((pl.col(f'mean_{target}') * pl.col('count') + global_mean * min_samples) / 
         (pl.col('count') + min_samples)).alias(f'{col}_encoded')
    ])
    
    # Select only the column name and encoded value
    cat_stats = cat_stats.select([pl.col(col), pl.col(f'{col}_encoded')])
    
    # Join back to the main dataframe
    return df.join(cat_stats, on=col, how='left')

# Encode high-cardinality categorical features
for col in ['location_type', 'descriptor', 'open_data_channel_type']:
    df = smoothed_target_encoding(df, col, 'response_time_hours')

# ----------------------
# Interaction Features
# ----------------------

# Create interaction features between key variables
df = df.with_columns([
    (pl.col('complaint_type_frequency') * pl.col('avg_response_time_by_complaint')).alias('complaint_intensity'),
    (pl.col('is_weekend') * pl.col('agency_frequency')).alias('weekend_agency_interaction'),
    (pl.col('is_holiday') * pl.col('complaint_type_frequency')).alias('holiday_complaint_interaction')
])

# ----------------------
# Text Features (if needed)
# ----------------------

# Get complaint type word count
df = df.with_columns([
    pl.col('complaint_type').str.split(' ').arr.lengths().alias('complaint_type_word_count'),
    pl.col('descriptor').str.split(' ').arr.lengths().alias('descriptor_word_count')
])

# ----------------------
# Create Feature Matrix
# ----------------------

# List of features to keep (excluding the original categorical fields that we've encoded)
features_to_keep = [
    # Target
    'response_time_hours',
    
    # Temporal features
    'year', 'month', 'day', 'hour', 'minute', 'weekday', 'quarter',
    'day_of_year', 'week_of_year', 'period_of_day', 'season',
    'is_weekend', 'is_business_hours', 'is_rush_hour', 'is_holiday',
    
    # Spatial features
    'zip_region', 'latitude', 'longitude',
    'borough_manhattan', 'borough_brooklyn', 'borough_queens', 
    'borough_bronx', 'borough_staten_island',
    'distance_to_manhattan', 'distance_to_brooklyn', 'distance_to_queens',
    'distance_to_bronx', 'distance_to_staten_island', 'distance_to_nyc_center',
    
    # Frequency features
    'complaint_type_frequency', 'zip_frequency', 'agency_frequency',
    
    # Target encoded features
    'avg_response_time_by_complaint', 'avg_response_time_by_agency',
    'avg_response_time_by_agency_complaint',
    'location_type_encoded', 'descriptor_encoded', 'open_data_channel_type_encoded',
    
    # Interaction features
    'complaint_intensity', 'weekend_agency_interaction', 'holiday_complaint_interaction',
    
    # Text features
    'complaint_type_word_count', 'descriptor_word_count'
]

# Create the final feature matrix
feature_matrix = df.select(features_to_keep)

# Save the feature matrix as parquet file for efficient storage
feature_matrix.collect().write_parquet("nyc_311_features.parquet")

print("Feature engineering complete! Features saved to nyc_311_features.parquet")
print(f"Created {len(features_to_keep)} features from the original dataset")

In [ ]:
import polars as pl
import numpy as np
from datetime import datetime, timedelta
from typing import List

# Assuming df is your loaded dataframe
# df = pl.read_csv("nyc_311_data.csv", columns=columns_to_keep)

# ======================================================================
# 1. TEMPORAL FEATURES
# ======================================================================

def create_temporal_features(df: pl.DataFrame) -> pl.DataFrame:
    """
    Create temporal features from created_date
    """
    # First, ensure created_date is a datetime type
    df = df.with_columns([
        pl.col("created_date").str.to_datetime("%Y-%m-%d %H:%M:%S")
    ])
    
    # Extract basic temporal components
    df = df.with_columns([
        pl.col("created_date").dt.year().alias("year"),
        pl.col("created_date").dt.month().alias("month"),
        pl.col("created_date").dt.day().alias("day"),
        pl.col("created_date").dt.hour().alias("hour"),
        pl.col("created_date").dt.minute().alias("minute"),
        pl.col("created_date").dt.weekday().alias("day_of_week"),
        pl.col("created_date").dt.month().alias("month_num"),
        pl.col("created_date").dt.ordinal_day().alias("day_of_year"),
        pl.col("created_date").dt.week().alias("week_of_year"),
        pl.col("created_date").dt.quarter().alias("quarter")
    ])
    
    # Weekend flag (0 = weekday, 1 = weekend)
    df = df.with_columns([
        (pl.col("day_of_week") >= 5).cast(pl.Int8).alias("is_weekend")
    ])
    
    # Time of day categories
    df = df.with_columns([
        pl.when(pl.col("hour").is_between(0, 5)).then(pl.lit("night"))
        .when(pl.col("hour").is_between(6, 11)).then(pl.lit("morning"))
        .when(pl.col("hour").is_between(12, 17)).then(pl.lit("afternoon"))
        .otherwise(pl.lit("evening"))
        .alias("time_of_day")
    ])
    
    # Rush hour flag (7-9 AM or 4-6 PM on weekdays)
    df = df.with_columns([
        ((pl.col("day_of_week") < 5) & 
         ((pl.col("hour").is_between(7, 8)) | 
          (pl.col("hour").is_between(16, 18)))).cast(pl.Int8).alias("is_rush_hour")
    ])
    
    # Season
    df = df.with_columns([
        pl.when(pl.col("month_num").is_between(3, 5)).then(pl.lit("spring"))
        .when(pl.col("month_num").is_between(6, 8)).then(pl.lit("summer"))
        .when(pl.col("month_num").is_between(9, 11)).then(pl.lit("fall"))
        .otherwise(pl.lit("winter"))
        .alias("season")
    ])
    
    # Business hours flag (9 AM - 5 PM, Monday-Friday)
    df = df.with_columns([
        ((pl.col("day_of_week") < 5) & 
         (pl.col("hour").is_between(9, 16))).cast(pl.Int8).alias("is_business_hours")
    ])
    
    return df

# ======================================================================
# 2. SPATIAL FEATURES
# ======================================================================

def create_spatial_features(df: pl.DataFrame) -> pl.DataFrame:
    """
    Create spatial features from geographic information
    """
    # Check if latitude and longitude are available
    has_lat_lon = ("latitude" in df.columns) and ("longitude" in df.columns)
    
    if has_lat_lon:
        # Filter out invalid coordinates
        df = df.with_columns([
            pl.when((pl.col("latitude") == 0) | (pl.col("longitude") == 0))
            .then(None)
            .otherwise(pl.col("latitude"))
            .alias("latitude"),
            
            pl.when((pl.col("latitude") == 0) | (pl.col("longitude") == 0))
            .then(None)
            .otherwise(pl.col("longitude"))
            .alias("longitude")
        ])
        
        # Manhattan distance from city center (approximate location of City Hall)
        city_hall_lat = 40.7128
        city_hall_lon = -74.0060
        
        df = df.with_columns([
            (pl.abs(pl.col("latitude") - city_hall_lat) + 
             pl.abs(pl.col("longitude") - city_hall_lon)).alias("manhattan_distance_to_city_hall")
        ])
        
        # Haversine distance to city center
        # Note: For performance with large datasets, consider if this precision is necessary
        df = df.with_columns([
            (2 * 6371 * pl.arcsin(
                pl.sqrt(
                    pl.sin((pl.col("latitude") - city_hall_lat) * np.pi / 180 / 2) ** 2 +
                    pl.cos(city_hall_lat * np.pi / 180) * 
                    pl.cos(pl.col("latitude") * np.pi / 180) * 
                    pl.sin((pl.col("longitude") - city_hall_lon) * np.pi / 180 / 2) ** 2
                )
            )).alias("km_distance_to_city_hall")
        ])
    
    # Create borough encoding if available
    if "borough" in df.columns:
        # One-hot encode boroughs
        df = df.with_columns([
            (pl.col("borough") == "MANHATTAN").cast(pl.Int8).alias("is_manhattan"),
            (pl.col("borough") == "BRONX").cast(pl.Int8).alias("is_bronx"),
            (pl.col("borough") == "BROOKLYN").cast(pl.Int8).alias("is_brooklyn"),
            (pl.col("borough") == "QUEENS").cast(pl.Int8).alias("is_queens"),
            (pl.col("borough") == "STATEN ISLAND").cast(pl.Int8).alias("is_staten_island")
        ])
    
    # Create zip code features
    if "incident_zip" in df.columns:
        # Extract first digit of zip code (geographic region)
        df = df.with_columns([
            pl.col("incident_zip").cast(pl.Utf8).str.slice(0, 1).cast(pl.Int16).alias("zip_first_digit")
        ])
    
    return df

# ======================================================================
# 3. CATEGORICAL ENCODING
# ======================================================================

def encode_categorical_features(df: pl.DataFrame) -> pl.DataFrame:
    """
    Encode categorical features using various methods
    """
    # Create frequency encodings for high cardinality features
    categorical_cols = ["agency", "complaint_type", "descriptor", "location_type"]
    
    for col in categorical_cols:
        if col in df.columns:
            # Frequency encoding
            freq_counts = df.select(
                pl.col(col).alias("category"),
                pl.lit(1).alias("count")
            ).group_by("category").sum()
            
            # Join back to original dataframe
            df = df.join(
                freq_counts.rename({"category": col, "count": f"{col}_frequency"}),
                on=col,
                how="left"
            )
            
            # Normalize frequency (as percentage of total)
            total = freq_counts["count"].sum()
            df = df.with_columns([
                (pl.col(f"{col}_frequency") / total).alias(f"{col}_frequency_pct")
            ])
    
    # One-hot encode open_data_channel_type if it exists and has low cardinality
    if "open_data_channel_type" in df.columns:
        channel_counts = df.group_by("open_data_channel_type").count()
        
        # If cardinality is reasonable, do one-hot encoding
        if len(channel_counts) <= 10:  # Arbitrary threshold
            unique_channels = channel_counts["open_data_channel_type"].to_list()
            for channel in unique_channels:
                if channel and isinstance(channel, str):  # Ensure valid values
                    df = df.with_columns([
                        (pl.col("open_data_channel_type") == channel).cast(pl.Int8)
                        .alias(f"channel_{channel.lower().replace(' ', '_')}")
                    ])
    
    return df

# ======================================================================
# 4. INTERACTION FEATURES
# ======================================================================

def create_interaction_features(df: pl.DataFrame) -> pl.DataFrame:
    """
    Create interaction features between different variables
    """
    # Create agency-complaint type combination feature
    if "agency" in df.columns and "complaint_type" in df.columns:
        df = df.with_columns([
            (pl.col("agency") + "_" + pl.col("complaint_type")).alias("agency_complaint_combo")
        ])
        
        # Get frequency count for this combination
        combo_counts = df.select(
            pl.col("agency_complaint_combo").alias("category"),
            pl.lit(1).alias("count")
        ).group_by("category").sum()
        
        # Join back to original dataframe
        df = df.join(
            combo_counts.rename({"category": "agency_complaint_combo", "count": "agency_complaint_frequency"}),
            on="agency_complaint_combo",
            how="left"
        )
    
    # Temporal-spatial interactions
    if "is_weekend" in df.columns and "borough" in df.columns:
        # Create weekend-borough combination
        df = df.with_columns([
            (pl.col("is_weekend").cast(pl.Utf8) + "_" + pl.col("borough")).alias("weekend_borough_combo")
        ])
    
    # Time of day and complaint type interaction
    if "time_of_day" in df.columns and "complaint_type" in df.columns:
        df = df.with_columns([
            (pl.col("time_of_day") + "_" + pl.col("complaint_type")).alias("time_complaint_combo")
        ])
    
    return df

# ======================================================================
# 5. AGGREGATION FEATURES
# ======================================================================

def create_aggregation_features(df: pl.DataFrame) -> pl.DataFrame:
    """
    Create statistical aggregation features
    Note: These operations can be expensive on large datasets
    """
    # Historical response time average by complaint type
    if "complaint_type" in df.columns and "response_time_hours" in df.columns:
        avg_response_by_complaint = df.select([
            pl.col("complaint_type"),
            pl.col("response_time_hours")
        ]).group_by("complaint_type").agg([
            pl.col("response_time_hours").mean().alias("avg_response_time"),
            pl.col("response_time_hours").std().alias("std_response_time"),
            pl.col("response_time_hours").count().alias("complaint_count")
        ])
        
        # Join back to original dataframe
        df = df.join(
            avg_response_by_complaint,
            on="complaint_type",
            how="left"
        )
    
    # Historical response time by agency
    if "agency" in df.columns and "response_time_hours" in df.columns:
        avg_response_by_agency = df.select([
            pl.col("agency"),
            pl.col("response_time_hours")
        ]).group_by("agency").agg([
            pl.col("response_time_hours").mean().alias("agency_avg_response"),
            pl.col("response_time_hours").count().alias("agency_request_count")
        ])
        
        # Join back to original dataframe
        df = df.join(
            avg_response_by_agency,
            on="agency",
            how="left"
        )
    
    # Complaints by zip code
    if "incident_zip" in df.columns:
        complaints_by_zip = df.select([
            pl.col("incident_zip"),
            pl.lit(1).alias("count")
        ]).group_by("incident_zip").sum()
        
        # Join back to original dataframe
        df = df.join(
            complaints_by_zip.rename({"count": "zip_total_complaints"}),
            on="incident_zip",
            how="left"
        )
    
    # Response times by day of week
    if "day_of_week" in df.columns and "response_time_hours" in df.columns:
        avg_by_day = df.select([
            pl.col("day_of_week"),
            pl.col("response_time_hours")
        ]).group_by("day_of_week").agg([
            pl.col("response_time_hours").mean().alias("day_avg_response")
        ])
        
        # Join back to original dataframe
        df = df.join(
            avg_by_day,
            on="day_of_week",
            how="left"
        )
    
    return df
